---
title: 'Lecture 2: Terminology, Baselines, Decision Trees'
description: Terminology, decision Trees
description-short: 'Supervised machine learning terminology: Features, target, examples, training, parameters and hyperparameters, Decision boundary, classification vs. regression, inference vs. prediction, accuracy vs. error, baselines, intuition of decision trees'
format:
  revealjs:
    slide-number: true
    smaller: true
    center: true
    logo: img/UBC-CS-logo.png
    resources:
      - data/
      - img/
---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
sys.path.append(os.path.join(os.path.abspath("."), "code"))
from plotting_functions import *
from IPython.display import HTML, display
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

plt.rcParams["font.size"] = 16
pd.set_option("display.max_colwidth", 200)
%matplotlib inline

DATA_DIR = 'data/' 

## Announcements 

- Things due this week 
    - Homework 1 (hw1): Due Sept 10 11:59pm 
- Homework 2 (hw2) has been released (Due: Sept 16, 11:59pm)
    - There is some autograding in this homework. 
- You can find the tentative due dates for all deliverables [here](https://ubc-cs.github.io/cpsc330-2024W1/README.html#deliverable-due-dates-tentative).
- Please monitor Piazza (especially pinned posts and instructor posts) for announcements. 
- I'll assume that you've watched the pre-lecture videos. 

## Recap: What is ML? 

- ML uses data to build models that find patterns, make predictions, or generate content.
- It helps computers learn from data to make decisions.
- No one model works for every situation.

## Recap: Supervised learning

- We wish to find a model function $f$ that relates $X$ to $y$.
- We use the model function to predict targets of new examples. 

![](img/sup-learning.png){.nostretch fig-align="center" width="700px"}

In the first part of this course, we'll focus on supervised machine learning. 

## Framework

- There are many frameworks to do do machine learning. 
- We'll mainly be using [`scikit-learn` framework](https://scikit-learn.org/stable/). 


In [ ]:
import IPython
url = "https://scikit-learn.org"
IPython.display.IFrame(width=1000, height=650, src=url)

## Running example 

Imagine you’re in the fortunate situation where, after graduating, you have a few job offers and need to decide which one to choose. You want to pick the job that will likely make you the happiest. To help with your decision, you collect data from like-minded people. Here are the first few rows of this toy dataset.


In [ ]:
#| echo: true
toy_happiness_df = pd.read_csv(DATA_DIR + 'toy_job_happiness.csv')
toy_happiness_df

# Terminology

## Features, target, example
- What are the **features** $X$? 
  - features = inputs = predictors = explanatory variables = regressors = independent variables = covariates 
- What's the target $y$?
  - target = output = outcome = response variable = dependent variable = labels 
- Can you think of other relevant features for this problem? 
- What is an example?

## Classification vs. Regression
- Is this a **classification** problem or a **regression** problem?  


In [ ]:
toy_happiness_df

## Prediction vs. Inference
- **Inference** is using the model to understand the relationship between the features and the target 
  - Why certain factors influence happiness? 
- **Prediction** is using the model to predict the target value for new examples based on learned patterns.
- Of course these goals are related, and in many situations we need both. 

## Training 
- In supervised ML, the goal is to learn a function that maps input features ($X$) to a target ($y$).
- The relationship between $X$ and $y$ is often complex, making it difficult to  define mathematically.
- We use algorithms to approximate this complex relationship between $X$ and $y$.
- **Training** is the process of applying an algorithm to learn the best function (or model) that maps $X$ to $y$. 
- In this course, I'll help you develop an intuition for how these models work and demonstrate how to use them in a machine learning pipeline.

## Separating $X$ and $y$

- In order to train a model we need to separate $X$ and $y$ from the dataframe. 


In [ ]:
#| echo: true
X = toy_happiness_df.drop(columns=["happy?"]) # Extract the feature set by removing the target column "happy?"
y = toy_happiness_df["happy?"] # Extract the target variable "happy?"

## Baseline
- Let's try a simplest algorithm of predicting the most popular target! 


In [ ]:
#| echo: true
from sklearn.dummy import DummyClassifier
model = DummyClassifier(strategy="most_frequent") # Initialize the DummyClassifier to always predict the most frequent class
model.fit(X, y) # Train the model on the feature set X and target variable y
toy_happiness_df['dummy_predictions'] = model.predict(X) # Add the predicted values as a new column in the dataframe
toy_happiness_df

# Decision trees 

## Intuition
- Decision trees find the "best" way to split data to make predictions.
- Each split is based on a question, like 'Are the colleagues supportive?'
- The goal is to group data by similar outcomes at each step.
- Now, let's see a decision tree using sklearn.

## Decision tree with `sklearn`
Let's train a simple decision tree on our toy dataset.  


In [ ]:
#| echo: true
from sklearn.tree import DecisionTreeClassifier # import the classifier
from sklearn.tree import plot_tree

model = DecisionTreeClassifier(max_depth=2, random_state=1) # Create a class object
model.fit(X, y)
plot_tree(model, filled=True, feature_names = X.columns, class_names=["Happy", "Unhappy"], impurity = False, fontsize=12);

## Prediction 
- Given a new example, how does a decision tree predict the class of this example?  
- What would be the prediction for the example below using the tree above? 
  - supportive_colleagues = 1, salary = 60000, coffee_machine = 0, vegan_boss = 1,  


In [ ]:
plot_tree(model, filled=True, feature_names = X.columns, class_names = ["Happy", "Unhappy"], impurity = False, fontsize=12);

## Prediction with `sklearn`
- What would be the prediction for the example below using the tree above? 
  - supportive_colleagues = 1, salary = 60000, coffee_machine = 0, vegan_boss = 1,  


In [ ]:
#| echo: true
test_example = [[1, 60000, 0, 1]]
print("Model prediction: ", model.predict(test_example))
plot_tree(model, filled=True, feature_names = X.columns, class_names = ["Happy", "Unhappy"], impurity = False, fontsize=9);

## Training (high level)

- How many possible questions could we ask in this context?


In [ ]:
X

## Training (high level)
- Decision tree learning is a search process to find the "best" tree among many possible ones.
- We evaluate questions using measures like **information gain** or the **Gini index** to find the most effective split.
- At each step, we aim to split the data into groups with more certainty in their outcomes.

## Parameters vs. Hyperparameters 
- Parameters 
  - The questions (features and thresholds) used to split the data at each node.
  - Example: salary <= 75000 at the root node  
- Hyperparameters
  - Settings that control tree growth, like `max_depth`, which limits how deep the tree can go.

## Decision boundary with `max_depth=1`


In [ ]:
X_subset = X[["supportive_colleagues", "salary"]]
model = DecisionTreeClassifier(max_depth=1, random_state=1)
model.fit(X_subset.values, y)
plot_tree_decision_boundary_and_tree(
    model, X_subset, y, x_label="supportive_colleagues", y_label="salary", fontsize=12, class_names = ["Happy", "Unhappy"]
)

## Decision boundary with `max_depth=2`


In [ ]:
X_subset = X[["supportive_colleagues", "salary"]]
model = DecisionTreeClassifier(max_depth=2, random_state=1)
model.fit(X_subset.values, y)
plot_tree_decision_boundary_and_tree(
    model, X_subset, y, x_label="supportive_colleagues", y_label="salary", fontsize=12, class_names = ["Happy", "Unhappy"]
)

## iClicker 2.2: Supervised vs unsupervised

Clicker cloud join link: https://join.iclicker.com/VYFJ

Select all of the following statements which are examples of supervised machine learning

- (A) Finding groups of similar properties in a real estate data set.
- (B) Predicting whether someone will have a heart attack or not on the basis of demographic, diet, and clinical measurement.
- (C) Grouping articles on different topics from different news sources (something like the Google News app).
- (D) Detecting credit card fraud based on examples of fraudulent and non-fraudulent transactions.
- (E) Given some measure of employee performance, identify the key factors which are likely to influence their performance.

## iClicker 2.3: Classification vs. Regression

Clicker cloud join link: https://join.iclicker.com/VYFJ
 
Select all of the following statements which are examples of regression problems

- (A) Predicting the price of a house based on features such as number of bedrooms and the year built.
- (B) Predicting if a house will sell or not based on features like the price of the house, number of rooms, etc.
- (C) Predicting percentage grade in CPSC 330 based on past grades.
- (D) Predicting whether you should bicycle tomorrow or not based on the weather forecast.
- (E) Predicting appropriate thermostat temperature based on the wind speed and the number of people in a room.

## iClicker 2.5: Baselines and Decision trees

iClicker cloud join link: https://join.iclicker.com/VYFJ

Select all of the following statements which are TRUE.

- (A) Change in features (i.e., binarizing features above) would change DummyClassifier predictions.
- (B) predict takes only X as argument whereas fit and score take both X and y as arguments.
- (C) For the decision tree algorithm to work, the feature values must be binary.
- (D) The prediction in a decision tree works by routing the example from the root to the leaf.


## HW2 Worksheet portion 
